In [1]:
import pickle
import sqlalchemy

from time import time
from tqdm import tqdm
from sqlalchemy import Table, Column, Integer, String, ForeignKey, Binary

In [2]:
def connect(user, password, db, host='localhost', port=5432):
    '''Returns a connection and a metadata object'''
    
    url = 'postgresql://{}:{}@{}:{}/{}'
    url = url.format(user, password, host, port, db)

    try:
        con = sqlalchemy.create_engine(url, client_encoding='utf8',  executemany_mode='batch')
        meta = sqlalchemy.MetaData(bind=con, reflect=True)
    except Exception as e:
        if "does not exist" in str(e):
            url = 'postgresql://{}:{}@{}:{}/template1'
            url = url.format(user, password, host, port)
            con = sqlalchemy.create_engine(url, client_encoding='utf8')
            meta = sqlalchemy.MetaData(bind=con, reflect=True)
        else:
            raise e
        
    return con, meta


con, meta = connect('postgres', 'ganga', 'jobs')
con.execute("""DROP TABLE IF EXISTS blobs""")
con.execute("""DROP TABLE IF EXISTS jobs""")

/home/needshelp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SADeprecationWarning: The MetaData.reflect flag is deprecated and will be removed in a future release.   Please use the MetaData.reflect() method.
  if __name__ == '__main__':
/home/needshelp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SADeprecationWarning: The MetaData.reflect flag is deprecated and will be removed in a future release.   Please use the MetaData.reflect() method.
  from ipykernel import kernelapp as app


In [3]:
size = 10000
JOBS = Table(
         f"jobs", meta,
         Column("jid", Integer, primary_key=True),
         Column("status", String),
         Column("name", String),
         Column("subjobs", Integer),
         Column("application", String),
         Column("backend", String),
         Column("backend_actualCE", String),
         Column("comment", String),
         extend_existing=True
    )

BLOBS = Table(
        f"blobs", meta,
        Column("jid", Integer, ForeignKey("jobs.jid")),
        Column("inputsandbox", Binary),
        Column("outputsandbox", Binary),
        Column("info", Binary),
        Column("comment", Binary),
        Column("time", Binary),
        Column("application", Binary),
        Column("backend", Binary),
        Column("inputfiles", Binary),
        Column("outputfiles", Binary),
        Column("non_copyable_outputfiles", Binary),
        Column("id", Binary),
        Column("status", Binary),
        Column("name", Binary),
        Column("inputdir", Binary),
        Column("outputdir", Binary),
        Column("inputdata", Binary),
        Column("outputdata", Binary),
        Column("splitter", Binary),
        Column("subjobs", Binary),
        Column("master", Binary),
        Column("postprocessors", Binary),
        Column("virtualization", Binary),
        Column("merger", Binary),
        Column("do_auto_resubmit", Binary),
        Column("metadata", Binary),
        Column("been_queued", Binary),
        Column("parallel_submit", Binary),
        extend_existing=True
    )

meta.create_all(con)

In [4]:
import copy
jobs = pickle.load(open("../../data/rows.pkl", "rb"))[:size]
rows = []
for itr, job in enumerate(jobs):
    row = {}
    for header, value in zip(
        ['jid', 'status', 'name', 'subjobs', 'application',
        'backend', 'backend_actualCE', 'comment'],
        job
    ):
        row[header] = value

    row["jid"] = itr
    rows.append(row)
jobs = rows


_blobs = pickle.load(open("../../data/blobs.pkl", "rb"))[:size]
blobs = []
for i, blo in enumerate(_blobs):
    # FIXME: Dirty patch, for some reason any change to a single element fof blobs array, goes to all the elements. I decided to create copies after failing to debug this at that time.
    blob = copy.deepcopy(blo)
    blob["jid"] = i
    blobs.append(blob)

In [14]:
try:
    insert_job = JOBS.insert().values(row)
    con.execute(insert_job)
except Exception as e:
    print(e)

'int' object has no attribute 'type'


In [8]:
row

{'jid': 0,
 'status': "'completed'",
 'name': 'None',
 'subjobs': 1,
 'application': "'85c74d55153c2b535aa07b2067c9381e'",
 'backend': "'Done'",
 'backend_actualCE': "'LCG.CNAF.it'",
 'comment': "''",
 0: 1}

In [6]:
        with tqdm(total=len(jobs)) as progress:
            progress.set_description("inserting jobs")
            for i, row in enumerate(jobs):
                row[0] = i+1
                insert_job = JOBS.insert().values(row)
                con.execute(insert_job)
                progress.update(1)

inserting jobs:   0%|          | 0/10000 [00:00<?, ?it/s]


AttributeError: 'int' object has no attribute 'type'

In [26]:
insert_blob = BLOBS.insert().values(row)
con.execute(insert_blob)

IntegrityError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "blobs" violates foreign key constraint "blobs_jid_fkey"
DETAIL:  Key (jid)=(0) is not present in table "jobs".

[SQL: INSERT INTO blobs (jid, inputsandbox, outputsandbox, info, comment, time, application, backend, inputfiles, outputfiles, non_copyable_outputfiles, id, status, name, inputdir, outputdir, inputdata, outputdata, splitter, subjobs, master, postprocessors, virtualization, merger, do_auto_resubmit, metadata, been_queued, parallel_submit) VALUES (%(jid)s, %(inputsandbox)s, %(outputsandbox)s, %(info)s, %(comment)s, %(time)s, %(application)s, %(backend)s, %(inputfiles)s, %(outputfiles)s, %(non_copyable_outputfiles)s, %(id)s, %(status)s, %(name)s, %(inputdir)s, %(outputdir)s, %(inputdata)s, %(outputdata)s, %(splitter)s, %(subjobs)s, %(master)s, %(postprocessors)s, %(virtualization)s, %(merger)s, %(do_auto_resubmit)s, %(metadata)s, %(been_queued)s, %(parallel_submit)s)]
[parameters: {'jid': 0, 'inputs